In [1]:
import numpy as np
import pandas as pd
import cv2
from sldata import SessionData

### Load Bonsai Data (Now with SLEAP) 

In [3]:
mouse_id = "7011"
session_id = "m5"
experiment = "clickbait-motivate"

print("=== Testing SessionData Class ===")
print(f"Loading data for {mouse_id}_{session_id}...")

data = SessionData(
    mouse_id=mouse_id,
    session_id=session_id,
    experiment=experiment,
    min_spikes=100,
    verbose=True)

=== Testing SessionData Class ===
Loading data for 7011_m5...
Successfully loaded spike_times from S:\clickbait-motivate\kilosorted\7011\m5\spike_times.npy
Successfully loaded spike_templates from S:\clickbait-motivate\kilosorted\7011\m5\spike_templates.npy
Successfully loaded templates from S:\clickbait-motivate\kilosorted\7011\m5\templates.npy
Successfully loaded sniff from S:\clickbait-motivate\preprocessed\7011\m5\sniff.npy
Loaded data: ['spike_times', 'spike_templates', 'templates', 'sniff']
Successfully loaded events from S:\clickbait-motivate\bonsai\7011\m5\events.csv
Video properties: 888x1968, 30.0 FPS, 81840 frames
Found 18627 peaks in sniff signal
Peak times range: 1.5 - 91611.5 ms
Loaded 56 clusters for 7011_m5
Loaded sniff data: 2748440 samples
Found 18627 sniff events


In [4]:
data.events[-5:]

,trial_number,timestamp,poke_left,poke_right,bonsai_centroid_x,bonsai_centroid_y,target_cell,iti,water_left,water_right,reward_state,click,flip_state,speed,direction,drinking,timestamp_ms
81832,39,2025-10-14 13:20:36.295756800,False,False,263,1317,NaN,False,True,False,False,False,False,4.000000,1.570796,False,2741174
81833,39,2025-10-14 13:20:36.328473600,False,False,267,1323,NaN,False,True,False,False,False,False,7.211102,0.982794,False,2741207
81834,39,2025-10-14 13:20:36.362022400,False,False,275,1323,NaN,False,True,False,False,False,False,8.000000,0.000000,False,2741241
81835,39,2025-10-14 13:20:36.393728000,False,False,275,1323,NaN,False,True,False,False,False,False,0.000000,0.000000,False,2741274
81836,39,2025-10-14 13:20:36.428147200,False,False,275,1323,NaN,False,True,False,False,False,False,0.000000,0.000000,False,2741307


### Visualize SLEAP Tracking

In [6]:
# Visualization options
scale = 0.5  # Scaling factor for preview animation
save = False # Toggle preview
video = True  # Toggle video background
bonsai_centroid = True  # Centroid from realtime tracking

# SLEAP Nodes
# nose = data.events[['nose.x', 'nose.y']].to_numpy().astype(np.int16)
# cent = data.events[['centroid.x', 'centroid.y']].to_numpy().astype(np.int16)
# base = data.events[['tailbase.x', 'tailbase.y']].to_numpy().astype(np.int16)
bon_cent = data.events[['bonsai_centroid_x', 'bonsai_centroid_y']].to_numpy().astype(np.int16)


for ii in range(0, data.total_frames):
    if video:
        data.video.set(cv2.CAP_PROP_POS_FRAMES, ii + data.video_offset)  # Get current video frame
        ret, canvas = data.video.read()
    else:
        canvas = np.zeros((data.height,data.width,3), dtype=np.uint8)  # Clear canvas
    
    # Draw Bonsai centroid
    if bonsai_centroid:
        cv2.circle(img=canvas, center=bon_cent[ii], radius=12, color=(128,128,128), thickness=-1, lineType=cv2.LINE_AA)

    # Draw SLEAP nodes
    # cv2.circle(img=canvas, center=nose[ii], radius=5, color=(255,0,0), thickness=-1, lineType=cv2.LINE_AA)  # Nose
    # cv2.circle(img=canvas, center=cent[ii], radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)  # Centroid
    # cv2.circle(img=canvas, center=base[ii], radius=5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)  # Tail base

    canvas = cv2.resize(canvas, dsize=(int(data.width*scale), int(data.height*scale)))
    canvas = cv2.rotate(canvas, cv2.ROTATE_90_CLOCKWISE) 

    if save:
        cv2.imwrite(f"S:/track-test-video/{mouse_id}_{session_id}_tracking_{ii}.png", canvas)
    else:
        cv2.imshow('SLEAP Test', canvas)
        # Wait for key press with timeout
        key = cv2.waitKey(16) & 0xFF
        # Break on 'q' key or ESC key
        if key == ord('q') or key == 27:
            break

cv2.waitKey(0)
cv2.destroyAllWindows()